In [3]:
import requests
import json
import fileinput
import yesg
from yahooquery import Screener
from yahooquery import Ticker


In [5]:
#initial setup code

#import google maps api
for line in fileinput.input(files = "config.env"):
    key = line
    
#sectors for pulling environmental scores
sectors = ['ms_energy',
           'oil_gas_drilling',
 'oil_gas_e_p',
 'oil_gas_equipment_services',
 'oil_gas_integrated',
 'oil_gas_midstream',
 'oil_gas_refining_marketing']

In [ ]:
def get_gas_stations():
    #This wil get the gas stations around the longitude and latitude
    type = "gas%20station" #%20 is escape for space
    latitude = "37.95098545221008"
    longitude = "-91.77840922372175"

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" + latitude + "%2C" + longitude + "&type=" + type + "&keyword=" + type + "&rankby=distance" + "&key=" + key
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    response_string = json.loads(response.text)

    gas_stations = []
    for gas_station in response_string['results']:
        gas_stations.append(gas_station['name'])
    return gas_stations


In [ ]:
near_stations = get_gas_stations()
print(get_gas_stations())

In [36]:
#uses the yesg library to turn a ticker into the latest esg score as a float
def latest_environmental_score(_ticker):
    try:
        cell = yesg.get_esg_short(str(_ticker))['E-Score'][0]
        return float(cell)
    except:
        return 0

In [65]:
latest_environmental_score('XOM')

18.5

In [29]:
s = Screener()

name = []
symbol = []
for n in range(len(sectors)):
    names = s.get_screeners(sectors[n],max)
    for m in range(len(names[sectors[n]]['quotes'])):
        name.append(names[sectors[n]]['quotes'][m]['longName'])
        symbol.append(names[sectors[n]]['quotes'][m]['symbol'])
        
company_dict = dict(zip(name,symbol))

    
station_dict = {'ZX': '',
                    'MPC #1043': 'MPC',
                    'MPC #1042': 'MPC',
                    'Mobil': 'XOM',
                    'On The Run': 'XOM',
                    'Moto Mart': '',
                    'MFA Oil': '',
                    'Midwest Petroleum': '',
                    'Sinclair': 'DINO',
                    "Casey's": 'CASY',
                    'Phillips 66': 'PSX',
                    'Exxon': 'XOM'}

gas_station_dict = company_dict | station_dict


In [26]:
gas_station_dict

{'Exxon Mobil Corporation': 'XOM',
 'Chevron Corporation': 'CVX',
 'Shell plc': 'SHEL',
 'TotalEnergies SE': 'TTE',
 'ConocoPhillips': 'COP',
 'BP p.l.c.': 'BP',
 'Equinor ASA': 'EQNR',
 'Enbridge Inc.': 'ENB',
 'Petróleo Brasileiro S.A. - Petrobras': 'PBR-A',
 'Schlumberger Limited': 'SLB',
 'EOG Resources, Inc.': 'EOG',
 'Canadian Natural Resources Limited': 'CNQ',
 'Enterprise Products Partners L.P.': 'EPD',
 'Occidental Petroleum Corporation': 'OXY',
 'Marathon Petroleum Corporation': 'MPC',
 'Pioneer Natural Resources Company': 'PXD',
 'Eni S.p.A.': 'E',
 'Phillips 66': 'PSX',
 'Valero Energy Corporation': 'VLO',
 'Hess Corporation': 'HES',
 'Woodside Energy Group Ltd': 'WDS',
 'TC Energy Corporation': 'TRP',
 'Suncor Energy Inc.': 'SU',
 'Kinder Morgan, Inc.': 'KMI',
 'Noble Corporation Plc': 'NE',
 'Transocean Ltd.': 'RIG',
 'Helmerich & Payne, Inc.': 'HP',
 'Patterson-UTI Energy, Inc.': 'PTEN',
 'Seadrill Limited': 'SDRL',
 'Borr Drilling Limited': 'BORR',
 'Diamond Offshore Dr

In [30]:
def get_ticker_list(_company_list, _gas_station_dict = gas_station_dict):
    temp_list = []
    for items in _company_list:
        if items in _gas_station_dict:
            temp_list.append(_gas_station_dict[items])
    return temp_list
            

In [31]:
get_ticker_list(['Sinclair','Exxon', "Casey's", 'Phillips 66'])

['DINO', 'XOM', 'CASY', 'PSX']

In [68]:
def get_best_station(_stations_list):
    temp_list = get_ticker_list(_stations_list)
    score_list = []
    for items in range(len(temp_list)):
        score_list.append(latest_environmental_score(temp_list[items]))
    index = score_list.index(max(score_list))
    print('The most environmentaly friendly gas staion is: ' + _stations_list[index] + ' with a score of ' + str(latest_environmental_score(temp_list[index])))
        
    

In [69]:
get_best_station(['Sinclair','Exxon', "Casey's", 'Phillips 66'])

The most environmentaly friendly gas staion is: Exxon with a score of 18.5
